In [1]:
from __future__ import print_function
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

In [8]:
class opt:
    dataroot = "/home/semion/src/datasets/imagenet/clouds/"
    manualSeed = None
    cuda = True
    batchSize = 32
    imageSize = 128
    nz = 100
    ngf = 256
    ndf = 64
    niter = 100000
    lr = 0.00002
    beta1 = 0.6
    ngpu = 1
#     restart_epoch = 650
#     netG = 'netG_epoch_{}.pth'.format(restart_epoch)
#     netD = 'netD_epoch_{}.pth'.format(restart_epoch)
    restart_epoch = 0
    netG = ''
    netD = ''
    outf = '.'
    workers = 4
    


if opt.manualSeed is None:
    opt.manualSeed = random.randint(1, 10000)
print("Random Seed: ", opt.manualSeed)
random.seed(opt.manualSeed)
torch.manual_seed(opt.manualSeed)
if opt.cuda:
    torch.cuda.manual_seed_all(opt.manualSeed)

cudnn.benchmark = True

if torch.cuda.is_available() and not opt.cuda:
    print("WARNING: You have a CUDA device, so you should probably run with --cuda")


    # folder dataset
dataset = dset.ImageFolder(root=opt.dataroot,
                               transform=transforms.Compose([
                                   transforms.Scale(opt.imageSize),
                                   transforms.CenterCrop(opt.imageSize),
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                               ]))

dataloader = torch.utils.data.DataLoader(dataset, batch_size=opt.batchSize,
                                         shuffle=True, num_workers=int(opt.workers))

ngpu = int(opt.ngpu)
nz = int(opt.nz)
ngf = int(opt.ngf)
ndf = int(opt.ndf)
nc = 3

Random Seed:  7361


In [9]:
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)


class _netG(nn.Module):
    def __init__(self, ngpu):
        super(_netG, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(     nz, ngf * 16, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 16),
            nn.ReLU(True),
            # state size. (ngf*16) x 4 x 4
            nn.ConvTranspose2d(ngf * 16, ngf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 8 x 8
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 16 x 16 
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 32 x 32
            nn.ConvTranspose2d(ngf * 2,     ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 64 x 64
            nn.ConvTranspose2d(    ngf,      nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 128 x 128
        )


    def forward(self, input):
        if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)
        return output


netG = _netG(ngpu)
netG.apply(weights_init)
if opt.netG != '':
    netG.load_state_dict(torch.load(opt.netG))
print(netG)


class _netD(nn.Module):
    def __init__(self, ngpu):
        super(_netD, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is (nc) x 128 x 128
            nn.Conv2d(nc, ndf, 4, stride=2, padding=1, bias=False), 
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 64 x 64
            nn.Conv2d(ndf, ndf * 2, 4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 32 x 32
            nn.Conv2d(ndf * 2, ndf * 4, 4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 16 x 16 
            nn.Conv2d(ndf * 4, ndf * 8, 4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 8 x 8
            nn.Conv2d(ndf * 8, ndf * 16, 4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ndf * 16),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*16) x 4 x 4
            nn.Conv2d(ndf * 16, 1, 4, stride=1, padding=0, bias=False),
            nn.Sigmoid()
            # state size. 1
        )

    def forward(self, input):
        if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)

        return output.view(-1, 1).squeeze(1)


netD = _netD(ngpu)
netD.apply(weights_init)
if opt.netD != '':
    netD.load_state_dict(torch.load(opt.netD))
print(netD)

criterion = nn.BCELoss()

input = torch.FloatTensor(opt.batchSize, 3, opt.imageSize, opt.imageSize)
noise = torch.FloatTensor(opt.batchSize, nz, 1, 1)
fixed_noise = torch.FloatTensor(opt.batchSize, nz, 1, 1).normal_(0, 1)
label = torch.FloatTensor(opt.batchSize)
real_label = 1
fake_label = 0

if opt.cuda:
    netD.cuda()
    netG.cuda()
    criterion.cuda()
    input, label = input.cuda(), label.cuda()
    noise, fixed_noise = noise.cuda(), fixed_noise.cuda()

fixed_noise = Variable(fixed_noise)

# setup optimizer
optimizerD = optim.Adam(netD.parameters(), lr=opt.lr, betas=(opt.beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=opt.lr, betas=(opt.beta1, 0.999))
torch.save(fixed_noise, "fixed_noise.pth")

for epoch in range(opt.restart_epoch + 1, opt.niter):
    for i, data in enumerate(dataloader, 0):
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        # train with real
        netD.zero_grad()
        real_cpu, _ = data
        batch_size = real_cpu.size(0)
        
        if opt.cuda:
            real_cpu = real_cpu.cuda()
        
        input.resize_as_(real_cpu).copy_(real_cpu)
        input += (torch.randn(input.size()).cuda() * 0.1)
        label.resize_(batch_size).fill_(real_label)
        inputv = Variable(input)
        labelv = Variable(label)

        output = netD(inputv)
        errD_real = criterion(output, labelv)
        errD_real.backward()
        D_x = output.data.mean()

        # train with fake
        noise.resize_(batch_size, nz, 1, 1).normal_(0, 1)
        noisev = Variable(noise)
        fake = netG(noisev)
        labelv = Variable(label.fill_(fake_label))
        output = netD(fake.detach())
        errD_fake = criterion(output, labelv)
        errD_fake.backward()
        D_G_z1 = output.data.mean()
        errD = errD_real + errD_fake
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        labelv = Variable(label.fill_(real_label))  # fake labels are real for generator cost
        output = netD(fake)
        errG = criterion(output, labelv)
        errG.backward()
        D_G_z2 = output.data.mean()
        optimizerG.step()

        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f'
              % (epoch, opt.niter, i, len(dataloader),
                 errD.data[0], errG.data[0], D_x, D_G_z1, D_G_z2))
        if i % 200 == 0:
            vutils.save_image(real_cpu,
                    '%s/real_samples.png' % opt.outf,
                    normalize=True)
            fake = netG(fixed_noise)
            vutils.save_image(fake.data,
                    '%s/fake_samples_epoch_%05d.png' % (opt.outf, epoch),
                    normalize=True)
    
    if epoch % 50 == 0:
    # do checkpointing
        torch.save(netG.state_dict(), '%s/netG_epoch_%d.pth' % (opt.outf, epoch))
        torch.save(netD.state_dict(), '%s/netD_epoch_%d.pth' % (opt.outf, epoch))

_netG (
  (main): Sequential (
    (0): ConvTranspose2d(100, 4096, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(4096, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU (inplace)
    (3): ConvTranspose2d(4096, 2048, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True)
    (5): ReLU (inplace)
    (6): ConvTranspose2d(2048, 1024, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True)
    (8): ReLU (inplace)
    (9): ConvTranspose2d(1024, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
    (11): ReLU (inplace)
    (12): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (14): ReLU (inplace)
    (15): ConvTranspose2d(256, 3, kernel_

/home/semion/anaconda3/envs/pytorch/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 4718592 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/home/semion/anaconda3/envs/pytorch/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 2284257280 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))


[651/100000][7/37] Loss_D: 0.1646 Loss_G: 3.3489 D(x): 0.9516 D(G(z)): 0.0979 / 0.0689
[651/100000][8/37] Loss_D: 0.1495 Loss_G: 3.5397 D(x): 0.9484 D(G(z)): 0.0831 / 0.0565
[651/100000][9/37] Loss_D: 0.1513 Loss_G: 3.4778 D(x): 0.9605 D(G(z)): 0.0968 / 0.0578
[651/100000][10/37] Loss_D: 0.1208 Loss_G: 3.5864 D(x): 0.9626 D(G(z)): 0.0738 / 0.0443
[651/100000][11/37] Loss_D: 0.1373 Loss_G: 3.6895 D(x): 0.9316 D(G(z)): 0.0589 / 0.0402
[651/100000][12/37] Loss_D: 0.0938 Loss_G: 3.8473 D(x): 0.9808 D(G(z)): 0.0673 / 0.0420
[651/100000][13/37] Loss_D: 0.1286 Loss_G: 4.2109 D(x): 0.9174 D(G(z)): 0.0331 / 0.0285
[651/100000][14/37] Loss_D: 0.1389 Loss_G: 3.6836 D(x): 0.9090 D(G(z)): 0.0376 / 0.0380
[651/100000][15/37] Loss_D: 0.2017 Loss_G: 3.2527 D(x): 0.8782 D(G(z)): 0.0601 / 0.0631
[651/100000][16/37] Loss_D: 0.2690 Loss_G: 3.0950 D(x): 0.8246 D(G(z)): 0.0580 / 0.0779
[651/100000][17/37] Loss_D: 0.2085 Loss_G: 2.6272 D(x): 0.9425 D(G(z)): 0.1286 / 0.0998
[651/100000][18/37] Loss_D: 0.1839 

/home/semion/anaconda3/envs/pytorch/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 4718592 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/home/semion/anaconda3/envs/pytorch/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 2284257280 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))


[652/100000][25/37] Loss_D: 0.0991 Loss_G: 3.8022 D(x): 0.9421 D(G(z)): 0.0364 / 0.0322
[652/100000][26/37] Loss_D: 0.0927 Loss_G: 3.9368 D(x): 0.9477 D(G(z)): 0.0353 / 0.0331
[652/100000][27/37] Loss_D: 0.1335 Loss_G: 3.4784 D(x): 0.9458 D(G(z)): 0.0700 / 0.0516
[652/100000][28/37] Loss_D: 0.1192 Loss_G: 3.5391 D(x): 0.9598 D(G(z)): 0.0711 / 0.0472
[652/100000][29/37] Loss_D: 0.1308 Loss_G: 3.7149 D(x): 0.9055 D(G(z)): 0.0283 / 0.0300
[652/100000][30/37] Loss_D: 0.1643 Loss_G: 3.1527 D(x): 0.9313 D(G(z)): 0.0813 / 0.0600
[652/100000][31/37] Loss_D: 0.1244 Loss_G: 3.3853 D(x): 0.9633 D(G(z)): 0.0797 / 0.0504
[652/100000][32/37] Loss_D: 0.1995 Loss_G: 3.7822 D(x): 0.8572 D(G(z)): 0.0360 / 0.0440
[652/100000][33/37] Loss_D: 0.1529 Loss_G: 3.0732 D(x): 0.9210 D(G(z)): 0.0619 / 0.0609
[652/100000][34/37] Loss_D: 0.1233 Loss_G: 3.3761 D(x): 0.9860 D(G(z)): 0.0955 / 0.0523
[652/100000][35/37] Loss_D: 0.1427 Loss_G: 3.5109 D(x): 0.9758 D(G(z)): 0.1021 / 0.0436
[652/100000][36/37] Loss_D: 2.55

...

[985/100000][26/37] Loss_D: 0.0498 Loss_G: 4.2932 D(x): 0.9902 D(G(z)): 0.0383 / 0.0206
[985/100000][27/37] Loss_D: 0.0525 Loss_G: 4.3207 D(x): 0.9824 D(G(z)): 0.0330 / 0.0178
[985/100000][28/37] Loss_D: 0.0581 Loss_G: 4.3398 D(x): 0.9739 D(G(z)): 0.0293 / 0.0172
[985/100000][29/37] Loss_D: 0.0821 Loss_G: 4.2864 D(x): 0.9532 D(G(z)): 0.0255 / 0.0162
[985/100000][30/37] Loss_D: 0.0718 Loss_G: 4.0941 D(x): 0.9675 D(G(z)): 0.0352 / 0.0206
[985/100000][31/37] Loss_D: 0.0597 Loss_G: 4.0988 D(x): 0.9776 D(G(z)): 0.0359 / 0.0202
[985/100000][32/37] Loss_D: 0.0621 Loss_G: 4.7448 D(x): 0.9666 D(G(z)): 0.0257 / 0.0165
[985/100000][33/37] Loss_D: 0.0429 Loss_G: 4.4379 D(x): 0.9964 D(G(z)): 0.0372 / 0.0194
[985/100000][34/37] Loss_D: 0.0658 Loss_G: 4.7815 D(x): 0.9619 D(G(z)): 0.0232 / 0.0142
[985/100000][35/37] Loss_D: 0.0273 Loss_G: 5.1387 D(x): 0.9880 D(G(z)): 0.0147 / 0.0103
[985/100000][36/37] Loss_D: 5.1192 Loss_G: 7.2298 D(x): 0.0064 D(G(z)): 0.0001 / 0.0008


/home/semion/anaconda3/envs/pytorch/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 4718592 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/home/semion/anaconda3/envs/pytorch/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 2284257280 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))


[986/100000][0/37] Loss_D: 0.0744 Loss_G: 2.3838 D(x): 0.9902 D(G(z)): 0.0595 / 0.1178
[986/100000][1/37] Loss_D: 0.5079 Loss_G: 4.1030 D(x): 0.9989 D(G(z)): 0.3851 / 0.0271
[986/100000][2/37] Loss_D: 0.2072 Loss_G: 6.0367 D(x): 0.9992 D(G(z)): 0.1694 / 0.0045
[986/100000][3/37] Loss_D: 0.0546 Loss_G: 7.0222 D(x): 0.9963 D(G(z)): 0.0444 / 0.0025
[986/100000][4/37] Loss_D: 0.0383 Loss_G: 5.9209 D(x): 0.9903 D(G(z)): 0.0276 / 0.0043
[986/100000][5/37] Loss_D: 0.0601 Loss_G: 5.4575 D(x): 0.9904 D(G(z)): 0.0470 / 0.0085
[986/100000][6/37] Loss_D: 0.0774 Loss_G: 4.7610 D(x): 0.9791 D(G(z)): 0.0506 / 0.0119
[986/100000][7/37] Loss_D: 0.0716 Loss_G: 4.9795 D(x): 0.9880 D(G(z)): 0.0554 / 0.0118
[986/100000][8/37] Loss_D: 0.2174 Loss_G: 5.4854 D(x): 0.9768 D(G(z)): 0.1653 / 0.0064
[986/100000][9/37] Loss_D: 0.0547 Loss_G: 6.4503 D(x): 0.9795 D(G(z)): 0.0320 / 0.0026
[986/100000][10/37] Loss_D: 0.0310 Loss_G: 7.4798 D(x): 0.9749 D(G(z)): 0.0040 / 0.0017
[986/100000][11/37] Loss_D: 0.3147 Loss_G:

Process Process-5337:
Process Process-5339:
Process Process-5340:
Process Process-5338:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/semion/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/semion/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/semion/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/semion/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/semion/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 34, in _worker_loop
    r = index_queue.get()
  File "/home/semion/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/queues.py", lin

KeyboardInterrupt: 